In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 186 kB/s             


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (224, 224)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


#model_1 = create_model_V1()
#model_1.summary()

In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [11]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [12]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        #print("hi")
        #print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator,
                class_weight = weights,
                verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        graph_dir = graph_dir+'/'+model_method.__name__
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [14]:
#model_test(5, 1, create_model_V1, (48,48), 'rgb')

In [15]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (224,224,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(224,224,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 130
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

2022-04-20 16:12:27.740869: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 16:12:27.837415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 16:12:27.838180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 16:12:27.839341: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 0s 0us/step
175


In [16]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343 

In [17]:
model_test(5, 50, transfer_learning_model_resnet_50, (224,224,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175


2022-04-20 16:12:56.823456: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-04-20 16:13:01.097883: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


718/718 - 248s - loss: 2.3061 - accuracy: 0.3625 - val_loss: 1.6340 - val_accuracy: 0.4107

Epoch 00001: val_accuracy improved from -inf to 0.41066, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5


...

Epoch 2/50
718/718 - 69s - loss: 1.6217 - accuracy: 0.5100 - val_loss: 1.4704 - val_accuracy: 0.4857

Epoch 00002: val_accuracy improved from 0.41867 to 0.48572, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 3/50
718/718 - 69s - loss: 1.2297 - accuracy: 0.6226 - val_loss: 1.2798 - val_accuracy: 0.5583

Epoch 00003: val_accuracy improved from 0.48572 to 0.55834, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 4/50
718/718 - 69s - loss: 0.9089 - accuracy: 0.7186 - val_loss: 1.2768 - val_accuracy: 0.5970

Epoch 00004: val_accuracy improved from 0.55834 to 0.59700, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 5/50
718/718 - 69s - loss: 0.6447 - accuracy: 0.7991 - val_loss: 1.7336 - val_accuracy: 0.5618

E

225/225 [==============================] - 17s 76ms/step - loss: 2.4323 - accuracy: 0.6411
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 73s - loss: 2.2942 - accuracy: 0.3697 - val_loss: 1.4370 - val_accuracy: 0.4676

Epoch 00001: val_accuracy improved from -inf to 0.46761, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 2/50
718/718 - 69s - loss: 1.6366 - accuracy: 0.5071 - val_loss: 1.6350 - val_accuracy: 0.4431

Epoch 00002: val_accuracy did not improve from 0.46761
Epoch 3/50
718/718 - 70s - loss: 1.2660 - accuracy: 0.6098 - val_loss: 1.1848 - val_accuracy: 0.5765

Epoch 00003: val_accuracy improved from 0.46761 to 0.57645, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 4/50
718/718 - 69s - los

225/225 [==============================] - 17s 76ms/step - loss: 2.2032 - accuracy: 0.6388
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 73s - loss: 2.2531 - accuracy: 0.3714 - val_loss: 1.3989 - val_accuracy: 0.4936

Epoch 00001: val_accuracy improved from -inf to 0.49356, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 2/50
718/718 - 69s - loss: 1.6021 - accuracy: 0.5165 - val_loss: 1.2496 - val_accuracy: 0.5446

Epoch 00002: val_accuracy improved from 0.49356 to 0.54458, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 3/50
718/718 - 69s - loss: 1.2257 - accuracy: 0.6194 - val_loss: 1.4213 - val_accuracy: 0.5484

Epoch 00003: val_accuracy improved from 0.54458 to 0.54842, saving model to /kaggle/w

225/225 [==============================] - 20s 88ms/step - loss: 2.2145 - accuracy: 0.6374
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 74s - loss: 2.3019 - accuracy: 0.3673 - val_loss: 1.8358 - val_accuracy: 0.3761

Epoch 00001: val_accuracy improved from -inf to 0.37607, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 2/50
718/718 - 69s - loss: 1.6338 - accuracy: 0.5024 - val_loss: 1.2930 - val_accuracy: 0.5400

Epoch 00002: val_accuracy improved from 0.37607 to 0.53998, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 3/50
718/718 - 69s - loss: 1.2406 - accuracy: 0.6136 - val_loss: 1.5023 - val_accuracy: 0.5079

Epoch 00003: val_accuracy did not improve from 0.53998
Epoch 4/50
718/718 - 69s - los

225/225 [==============================] - 17s 75ms/step - loss: 2.1915 - accuracy: 0.6298


In [18]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (224,224,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(224,224,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 13
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

58900480/58889256 [==============================] - 0s 0us/step
19


In [19]:
transfer_model_vgg_16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 3591  

In [20]:
model_test(5, 50, transfer_learning_model_vgg_16, (224,224,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 75s - loss: 2.8643 - accuracy: 0.1854 - val_loss: 1.7960 - val_accuracy: 0.2924

Epoch 00001: val_accuracy improved from -inf to 0.29241, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 2/50
718/718 - 67s - loss: 2.3590 - accuracy: 0.2876 - val_loss: 1.6980 - val_accuracy: 0.3556

Epoch 00002: val_accuracy improved from 0.29241 to 0.35563, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 3/50
718/718 - 67s - loss: 2.0999 - accuracy: 0.3672 - val_loss: 1.6064 - val_accuracy: 0.3936

Epoch 00003: val_accuracy improved from 0.35563 to 0.39359, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_1.h

225/225 [==============================] - 19s 84ms/step - loss: 2.5354 - accuracy: 0.5961
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 68s - loss: 2.7417 - accuracy: 0.2100 - val_loss: 2.2753 - val_accuracy: 0.2226

Epoch 00001: val_accuracy improved from -inf to 0.22257, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 2/50
718/718 - 67s - loss: 2.2622 - accuracy: 0.3157 - val_loss: 1.8610 - val_accuracy: 0.3285

Epoch 00002: val_accuracy improved from 0.22257 to 0.32846, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 3/50
718/718 - 67s - loss: 1.9956 - accuracy: 0.3905 - val_loss: 1.5468 - val_accuracy: 0.4131

Epoch 00003: val_accuracy improved from 0.32846 to 0.41310, saving model to /kaggle/working/Saved_

225/225 [==============================] - 20s 91ms/step - loss: 1.6379 - accuracy: 0.5960
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 68s - loss: 2.7655 - accuracy: 0.1889 - val_loss: 1.9003 - val_accuracy: 0.2797

Epoch 00001: val_accuracy improved from -inf to 0.27969, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 2/50
718/718 - 67s - loss: 2.3152 - accuracy: 0.2827 - val_loss: 1.9099 - val_accuracy: 0.3123

Epoch 00002: val_accuracy improved from 0.27969 to 0.31226, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 3/50
718/718 - 67s - loss: 2.0339 - accuracy: 0.3754 - val_loss: 1.6083 - val_accuracy: 0.4284

Epoch 00003: val_accuracy improved from 0.31226 to 0.42842, saving model to /kaggle/working/Saved_

225/225 [==============================] - 18s 78ms/step - loss: 2.7951 - accuracy: 0.6049
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 68s - loss: 2.8548 - accuracy: 0.1753 - val_loss: 1.8382 - val_accuracy: 0.2710

Epoch 00001: val_accuracy improved from -inf to 0.27099, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 2/50
718/718 - 67s - loss: 2.3824 - accuracy: 0.2801 - val_loss: 1.9048 - val_accuracy: 0.3072

Epoch 00002: val_accuracy improved from 0.27099 to 0.30721, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 3/50
718/718 - 67s - loss: 2.1420 - accuracy: 0.3520 - val_loss: 1.6803 - val_accuracy: 0.3652

Epoch 00003: val_accuracy improved from 0.30721 to 0.36520, saving model to /kaggle/working/Saved_

225/225 [==============================] - 17s 75ms/step - loss: 2.9577 - accuracy: 0.5936
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 71s - loss: 2.7056 - accuracy: 0.2265 - val_loss: 2.1747 - val_accuracy: 0.2707

Epoch 00001: val_accuracy improved from -inf to 0.27068, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 2/50
718/718 - 67s - loss: 2.2387 - accuracy: 0.3275 - val_loss: 1.8168 - val_accuracy: 0.3597

Epoch 00002: val_accuracy improved from 0.27068 to 0.35969, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 3/50
718/718 - 67s - loss: 2.0084 - accuracy: 0.3920 - val_loss: 1.6666 - val_accuracy: 0.4097

Epoch 00003: val_accuracy improved from 0.35969 to 0.40968, saving model to /kaggle/working/Saved_

225/225 [==============================] - 18s 79ms/step - loss: 1.6004 - accuracy: 0.5939
